In [1]:
import sys, os
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib
import weatherbench2 as wb
import metpy
import copy
import wrf
import time
from metpy.units import units
import cartopy.crs as ccrs
print('modules loaded')
import datetime

modules loaded


In [2]:
figpath='/users/mfeldman/figs/'
#EVENT SELECTION ERA5
era5='gs://weatherbench2/datasets/era5/1959-2022-6h-1440x721.zarr'
xr_era5=xr.open_zarr(era5)
xr_era5=xr_era5.sel(time=xr_era5.time.dt.year.isin([2020]))
xr_era5=xr_era5.sel(time=xr_era5.time.dt.month.isin([4]))#np.arange(3,10)))
xr_era5=xr_era5.sel(time=xr_era5.time.dt.day.isin([12,13]))
zsurf_c=xr_era5['geopotential_at_surface']

plevel=copy.deepcopy(xr_era5.level.values)
        
xr_era5['pressure'] = (('level'), plevel)
plevel=xr_era5.pressure
xr_dataset = xr_era5.assign_coords(level=np.arange(len(plevel)))
tlevel=xr_era5.temperature
qlevel=xr_era5.specific_humidity
plevel_dim=np.ones([1,len(plevel),1,1])
zlevel=xr_era5.geopotential
ulevel=xr_era5.u_component_of_wind
vlevel=xr_era5.v_component_of_wind
for n1 in range(len(plevel.data)):
    plevel_dim[0,n1,0,0]=plevel[n1]
plevel_exp=(tlevel/tlevel)*plevel_dim
psurf=xr_era5.mean_sea_level_pressure
tsurf=xr_era5['2m_temperature']
zsurf = zsurf_c.expand_dims(dim={"time": psurf.time}, axis=0)

ps_era=psurf/100
zs=zsurf/9.81
zl=zlevel/9.81

era_cape=wrf.cape_2d(pres_hpa=plevel_exp, tkel=tlevel, qv=qlevel, height=zl, terrain=zs, psfc_hpa=ps_era, ter_follow=False)
era_srh=wrf.srhel(u=ulevel, v=vlevel, height=zl, terrain=zs, top=3000.0)

In [3]:
era_cape=era_cape.assign_coords(longitude=psurf.longitude.values);
era_cape=era_cape.assign_coords(latitude=psurf.latitude.values); era_cape=era_cape.assign_coords(time=psurf.time.values)
era_srh=era_srh.assign_coords(longitude=psurf.longitude.values);
era_srh=era_srh.assign_coords(latitude=psurf.latitude.values); era_srh=era_srh.assign_coords(time=psurf.time.values)

In [5]:
start=xr_era5.time[0]
end=xr_era5.time[-1]

#LOADING FORECAST DATASET
model='pangu'
#model='ifs'
latslice=slice(50,20)
#latslice=slice(20,50)
dataset='gs://weatherbench2/datasets/pangu/2018-2022_0012_0p25.zarr'
#dataset='gs://weatherbench2/datasets/hres/2016-2022-0012-1440x721.zarr'
days=[6,7,8,9,10,11,12]
hours=[0,12]
for day in days:
    for hour in hours:
        xr_dataset=xr.open_zarr(dataset)
        xr_dataset=xr_dataset.sel(time=xr_dataset.time.dt.year.isin([2020]))
        xr_dataset=xr_dataset.sel(time=xr_dataset.time.dt.month.isin([4]))
        xr_dataset=xr_dataset.sel(time=xr_dataset.time.dt.day.isin([day]))
        xr_dataset=xr_dataset.sel(time=xr_dataset.time.dt.hour.isin([hour]))
        fcst_init=xr_dataset.time[0]
        #print(fcst_init)
        init=fcst_init.dt.strftime('%Y%m%d%H').values; print(init)
        xr_dataset=xr_dataset.sel(prediction_timedelta=slice(start-fcst_init,end-fcst_init))
        xr_dataset = xr_dataset.sortby('level', ascending=False)
        
        plevel=copy.deepcopy(xr_dataset.level.values)
        
        xr_dataset['pressure'] = (('level'), plevel)
        plevel=xr_dataset.pressure
        xr_dataset = xr_dataset.assign_coords(level=np.arange(len(plevel)))
        
        tlevel=xr_dataset.temperature
        qlevel=xr_dataset.specific_humidity
        plevel_dim=np.ones([1,len(plevel),1,1])
        zlevel=xr_dataset.geopotential
        ulevel=xr_dataset.u_component_of_wind
        vlevel=xr_dataset.v_component_of_wind
        for n1 in range(len(plevel.data)):
            plevel_dim[0,n1,0,0]=plevel[n1]
        plevel_exp=(tlevel/tlevel)*plevel_dim
        
        psurf=xr_dataset.mean_sea_level_pressure
        tsurf=xr_dataset['2m_temperature']
        #print(zsurf_c.shape,zsurf_c.dims,psurf.dims)
        zsurf = zsurf_c.expand_dims(dim={"time": psurf.time}, axis=0)
        zsurf = zsurf.expand_dims(dim={"prediction_timedelta": psurf.prediction_timedelta}, axis=1)
        #print(zsurf.shape,psurf.shape)
        
        ps=psurf/100
        zs=zsurf/9.81
        zl=zlevel/9.81

        cape=wrf.cape_2d(pres_hpa=plevel_exp, tkel=tlevel, qv=qlevel, height=zl, terrain=zs, psfc_hpa=ps, ter_follow=False)
        srh=wrf.srhel(u=ulevel, v=vlevel, height=zl, terrain=zs, top=3000.0)

        cape=cape.assign_coords(longitude=psurf.longitude.values);
        cape=cape.assign_coords(latitude=psurf.latitude.values);
        cape=cape.assign_coords(prediction_timedelta=ps_era.time.values)
        #print(cape.mcape_mcin_lcl_lfc,cape.latitude,cape.time,psurf.latitude,psurf.time)
        cm=plt.cm.Reds
        cm.set_bad(color="grey")
        cm.set_under(color="grey")
        p = cape.sel(mcape_mcin_lcl_lfc='mcape',latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="prediction_timedelta",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=200,vmax=3000,levels=np.arange(200,3000,100),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        #f.set_figheight(3)
        f.savefig(figpath+model+init+'_cape.png',  dpi=300, bbox_inches='tight')
        plt.close()

        srh=srh.assign_coords(longitude=psurf.longitude.values);
        srh=srh.assign_coords(latitude=psurf.latitude.values);
        srh=srh.assign_coords(prediction_timedelta=ps_era.time.values)
        #print(cape.mcape_mcin_lcl_lfc,cape.latitude,cape.time,psurf.latitude,psurf.time)
        cm=plt.cm.Reds
        cm.set_bad(color="grey")
        cm.set_under(color="grey")
        p = srh.sel(latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="prediction_timedelta",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=0,vmax=1000,levels=np.arange(0,1000,50),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        #f.set_figheight(3)
        f.savefig(figpath+model+init+'_srh.png',  dpi=300, bbox_inches='tight')
        plt.close()



        cape=cape.squeeze(dim='time'); cape=cape.rename({'prediction_timedelta':'time'})
        srh=srh.squeeze(dim='time'); srh=srh.rename({'prediction_timedelta':'time'})
        cm=plt.cm.RdBu_r
        cm.set_bad(color="grey")
        dcape=cape.fillna(0)-era_cape.fillna(0)
        dsrh=srh.fillna(0)-era_srh.fillna(0)
        p = dcape.sel(mcape_mcin_lcl_lfc='mcape',latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=-1000,vmax=1000,levels=np.arange(-1000,1010,100),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_cape_d.png',  dpi=300, bbox_inches='tight')
        plt.close()


        cm=plt.cm.RdBu_r
        cm.set_bad(color="grey")
        p = dsrh.sel(latitude=latslice,longitude=slice(250,300)).plot(x="longitude", y="latitude", col="time",\
                         col_wrap=4,transform=ccrs.PlateCarree(), subplot_kws={'projection':ccrs.PlateCarree()},\
                         cmap=cm,vmin=-500,vmax=500,levels=np.arange(-500,505,50),extend='both')# col_wrap=4)
        for ax in p.axs.flat:
                        ax.coastlines()
        f=p.fig
        f.savefig(figpath+model+init+'_srh_d.png',  dpi=300, bbox_inches='tight')
        plt.close()

2020040600
2020040612
2020040700
2020040712
2020040800
2020040812
2020040900
2020040912
2020041000
2020041012
2020041100
2020041112
2020041200


ValueError: conflicting sizes for dimension 'prediction_timedelta': length 7 on <this-array> and length 8 on {'mcape_mcin_lcl_lfc': 'mcape_mcin_lcl_lfc', 'longitude': 'longitude', 'latitude': 'latitude', 'prediction_timedelta': 'prediction_timedelta', 'time': <this-array>}